# Advanced Lane Lines Detection

The goals / steps of this project are the following:

* Apply a perspective transform to rectify image ("birds-eye view").
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
from LaneDetector import *

%matplotlib inline

In [ ]:
test_img_paths = glob.glob('test_images/test*.jpg')

In [ ]:
def get_image(img_path, visualise=False):
    '''
    Load an image from the 'img_path' and pre-process it
    :param img_path (string): Image path
    :param visualise (boolean): Boolean flag for visualisation
    :return : Transformed Image, (PT matrix, PT inv matrix)
    '''
    img = mpimg.imread(img_path)
    return preprocess_image(img, visualise=visualise)

## 1. Perspective Transformation

In [ ]:
for path in test_img_paths[:]:
        get_image(path, visualise=True)

## 2. Generate Thresholded Binary image

In [ ]:
for img_path in test_img_paths[:2]:
        img, _ = get_image(img_path)
        get_binary_image(img, visualise=True)

## 3. Detect Lane Lines: Peaks in Histogram & Sliding Window Technique

In [ ]:
for img_path in test_img_paths[:]:
        laneDet = LaneDetector()
        img, _ = get_image(img_path)
        binary = get_binary_image(img, visualise=False)
        laneDet.polyfit_sliding_window(binary, visualise=True)

## 4. Detect Lane Lines: Adaptive Search 

In [ ]:
cap = cv2.VideoCapture('project_video.mp4')

videoframes = []

while True:
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    videoframes.append(frame)
    

In [ ]:
for img in videoframes[0: 5]:
    
    warped, (M, invM) = preprocess_image(img)
    laneDet = LaneDetector()
    
    max_attempts = 5
    
    if laneDet.reset == True:
        binary = get_binary_image(warped)
        ret, out, poly_param = laneDet.polyfit_sliding_window(binary, visualise=False, diagnostics=True)
        if ret:
            laneDet.reset = False
            laneDet.cache = np.array([poly_param])

    else:
        out, poly_param = laneDet.polyfit_adapt_search(warped, poly_param, visualise=False, diagnostics=False)
        if laneDet.attempts == max_attempts:
            laneDet.attempts = 0
            laneDetreset = True

    out_unwarped = cv2.warpPerspective(out, invM, (IMG_SHAPE[1], IMG_SHAPE[0]), flags=cv2.INTER_LINEAR)

    img_overlay = np.copy(img)
    img_overlay = cv2.addWeighted(out_unwarped, 0.5, img, 0.5, 0)

    plot_images([(warped, 'Original'), (out, 'Out'), (img_overlay, 'Overlay')], figsize=(20, 18))

## 7. Compute meters/pixel

In [ ]:
def compute_mppx(img, dashed_line_loc, visualise=False):
    '''
    Converts from pixel space to real world space and calculates the metres/pixel
    :param img (ndarray): Warped Image
    :param dashed_line_loc (string): Dashed line location (left/right)
    :param visualise (boolean): Boolean flag for visualisation
    : return (float32, float32): (metres/pixel y direction, metres/pixel x-direction)
    '''
    lane_width = 3.7
    dashed_line_len = 3.048
    
    if dashed_line_loc == 'left':
        y_top = 295
        y_bottom = 405
    elif dashed_line_loc == 'right':
        y_top = 395
        y_bottom = 495
        
    binary = get_binary_image(img)
    histogram = np.sum(binary[int(binary.shape[0] / 2):, :], axis=0)
    midpoint = int(histogram.shape[0] / 2)
    
    x_left = np.argmax(histogram[:midpoint])
    x_right = np.argmax(histogram[midpoint:]) + midpoint
    
    x_mppx = lane_width / (x_right - x_left)
    y_mppx = dashed_line_len / (y_bottom - y_top)
    
    if visualise:
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')
        
        if dashed_line_loc == 'left':
            plt.plot((x_left, x_left), (y_top, y_bottom), 'r')
            plt.text(x_left + 10, (y_top + y_bottom) // 2, '{} m'.format(dashed_line_len), color='r', fontsize=20)

        elif dashed_line_loc == 'right':
            plt.plot((x_right, x_right), (y_top, y_bottom), 'r')
            plt.text(x_right + 10, (y_top + y_bottom) // 2, '{} m'.format(dashed_line_len), color='r',fontsize=20)

        plt.plot((x_left, x_right), (img.shape[0] - 200 , img.shape[0] - 200), 'r')
        plt.text((x_left + x_right) // 2, img.shape[0] - 220, '{} m'.format(lane_width), color='r', fontsize=20)
        
    return y_mppx, x_mppx

In [ ]:
img, _ = get_image(test_img_paths[0])
y_mppx1, x_mppx1 = compute_mppx(img, dashed_line_loc='right', visualise=True)

img, _ = get_image(test_img_paths[1])
y_mppx2, x_mppx2 = compute_mppx(img, dashed_line_loc='left', visualise=True)

x_mppx = (x_mppx1 + x_mppx2) / 2
y_mppx = (y_mppx1 + y_mppx2) / 2

print('Average meter/px along x-axis: {:.4f}'.format(x_mppx))
print('Average meter/px along y-axis: {:.4f}'.format(y_mppx))

## 8. Compute Lane Line Curvature

In [ ]:
lanedet = LaneDetector()

for img in videoframes[1000: 1010]:

    warped, (M, invM) = preprocess_image(img)

    binary = get_binary_image(warped)
    ret, img_poly, poly_param = lanedet.polyfit_sliding_window(binary)

    left_curverad, right_curverad = lanedet.compute_curvature(poly_param, y_mppx, x_mppx)
    curvature = (left_curverad + right_curverad) / 2
    offset = lanedet.compute_offset_from_center(poly_param, x_mppx)
    result = lanedet.draw(img, warped, invM, poly_param, curvature, offset)

    plot_images([(img_poly, 'Polyfit'), (result, 'Result')])

## 9. Pipeline

In [ ]:
lanedet = LaneDetector()

for img in videoframes[0:15]:
    result = lanedet.pipeline(img, visualise=True, diagnostics=1)

In [ ]:
images1 = glob.glob('test_images/*.jpg')

lanedet = LaneDetector()

img = mpimg.imread(images1[0])

_ = lanedet.pipeline(img, visualise=True, diagnostics=True)

## 9. Process Video

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

lanedet = LaneDetector()

process_frame = lambda frame: lanedet.pipeline(frame, diagnostics=False)

video_output = 'harder_challenge_output.mp4'
video_input = VideoFileClip('harder_challenge_video.mp4')                          
#processed_video = video_input.fl_image(process_frame)
#%time processed_video.write_videofile(video_output, audio=False)

# video1_output = 'challenge_video_output.mp4'
# video1_input = VideoFileClip('challenge_video.mp4')#.subclip(4, 7)                          
# processed_video = video1_input.fl_image(process_frame)
# %time processed_video.write_videofile(video1_output, audio=False)

# video2_output = 'harder_challenge_video_output.mp4'
# video2_input = VideoFileClip('harder_challenge_video.mp4')                          
# processed_video = video2_input.fl_image(process_frame)#.subclip(0,5)
# %time processed_video.write_videofile(video2_output, audio=False)